导入包

In [1]:
import numpy as np
from scipy.io import loadmat,savemat
from sklearn.model_selection import train_test_split
import os
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn import preprocessing

导入数据

In [2]:
data = np.array([])
n = 0
target = np.array([])
for i,n in zip(os.listdir('9个类别的数据集-train/'),[2,3,5,6,8,10,11,12,14]):
    a,b = os.path.splitext(i)
    new = loadmat('9个类别的数据集-train/'+i)[a]
    for j in range(new.shape[0]):
        target = np.append(target,n)#为每个样本赋予标签
    data = np.append(data,new)
data = np.reshape(data,(-1,200))
target = np.reshape(target,(-1,1))

In [3]:
data

array([[3693., 4256., 4503., ..., 1050., 1024., 1024.],
       [2589., 4372., 4334., ..., 1050., 1024., 1019.],
       [3693., 4128., 4507., ..., 1045., 1029., 1019.],
       ...,
       [3192., 4235., 4313., ..., 1026., 1023., 1000.],
       [3192., 4111., 4313., ..., 1030., 1014., 1008.],
       [3192., 4115., 4313., ..., 1031., 1014., 1013.]])

数据预处理——标准化

In [4]:
scaler = preprocessing.StandardScaler().fit(data)
data = scaler.transform(data)

In [5]:
data

array([[ 1.93131339,  0.46936621,  0.58152419, ...,  0.42538707,
         0.29026057,  1.94286538],
       [-0.98136932,  1.01466346, -0.12126842, ...,  0.42538707,
         0.29026057,  1.22873286],
       [ 1.93131339, -0.13234109,  0.59815834, ...,  0.18687544,
         0.727489  ,  1.22873286],
       ...,
       [ 0.60952531,  0.37064861, -0.20859767, ..., -0.71946873,
         0.20281488, -1.48497071],
       [ 0.60952531, -0.21225534, -0.20859767, ..., -0.52865943,
        -0.58419628, -0.34235868],
       [ 0.60952531, -0.19345199, -0.20859767, ..., -0.48095711,
        -0.58419628,  0.37177384]])

划分训练集（80%）、验证集（20%）

In [6]:
train_x,dev_x,train_y,dev_y = train_test_split(data,target,test_size=0.2,random_state=30)
test_x = np.array(loadmat('data_test_final.mat')['data_test_final'])

In [7]:
test_x

array([[3158, 4122, 4145, ..., 1022, 1019, 1010],
       [3023, 3737, 4027, ..., 1019, 1008, 1004],
       [2743, 4388, 4591, ..., 1056, 1030, 1025],
       ...,
       [2561, 3881, 4051, ..., 1018,  998, 1015],
       [2745, 4391, 4335, ..., 1046, 1029, 1019],
       [3009, 3986, 3967, ..., 1011, 1009, 1000]], dtype=uint16)

特征降维

In [8]:
pca = PCA(n_components='mle',copy=False)
pca.fit_transform(train_x)
pca.transform(dev_x)
pca.transform(test_x)

array([[-1.25685758e+04, -2.99265760e+04,  1.58716721e+02, ...,
        -7.92734213e+01, -2.06196317e+01, -6.35105634e+01],
       [-5.66151442e+03, -3.98049563e+04, -1.62262249e+02, ...,
        -5.97618539e+01, -4.61844108e+00, -4.45135409e+01],
       [-1.45728094e+04, -3.37579225e+04,  3.28498687e+02, ...,
        -6.80936170e+01, -2.26249462e+01, -5.48671413e+01],
       ...,
       [-7.67225816e+03, -3.55382898e+04, -4.17928149e+01, ...,
        -5.09574647e+01, -7.83039649e+00, -3.81193351e+01],
       [-1.46714332e+04, -3.18373223e+04,  2.88894993e+02, ...,
        -7.93164844e+01, -3.73214215e+01, -5.73002016e+01],
       [-9.70278905e+03, -3.58526614e+04, -2.51543999e+02, ...,
        -6.32968321e+01, -2.18964486e+01, -5.74647029e+01]])

特征降维后，降成了195个特征

In [9]:
pca.explained_variance_ratio_.shape

(195,)

训练、预测

In [12]:
clf = SVC(gamma='auto',decision_function_shape='ovo')

格点搜索法搜索最优参数

In [37]:
from sklearn.model_selection import GridSearchCV
param_test ={"C":[100],
             "gamma":[0.001,0.005,0.006,0.007,0.01]}
gsearch= GridSearchCV(estimator =clf,param_grid =param_test,cv=10)
gsearch.fit(train_x,train_y.ravel())

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma=0.006, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [100], 'gamma': [0.001, 0.005, 0.006, 0.007, 0.01]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

输出SVC自带评分指标

In [41]:
clf = SVC(C=100,gamma=0.006,kernel= "rbf",decision_function_shape='ovo')
clf.fit(train_x,train_y.ravel())
pred_y = clf.predict(dev_x)
clf.score(dev_x,dev_y)

0.9545126353790614

In [35]:
pred_y

array([ 3.,  3., 10., ...,  5.,  6.,  3.])

输出评价指标

In [36]:
from sklearn.metrics import classification_report
print(classification_report(dev_y,pred_y,target_names = None))

              precision    recall  f1-score   support

         2.0       0.92      0.95      0.93       138
         3.0       0.94      0.94      0.94       372
         5.0       0.98      0.96      0.97       100
         6.0       0.99      0.99      0.99       168
         8.0       0.93      0.94      0.93       192
        10.0       0.95      0.90      0.92       142
        11.0       0.99      0.97      0.98        79
        12.0       0.98      1.00      0.99       128
        14.0       1.00      1.00      1.00        66

   micro avg       0.95      0.95      0.95      1385
   macro avg       0.96      0.96      0.96      1385
weighted avg       0.95      0.95      0.95      1385



使用K近邻分类

In [40]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=9)
neigh.fit(train_x,train_y.ravel())
print(neigh.score(dev_x,dev_y))

0.8028880866425993


In [41]:
from sklearn.metrics import classification_report
print(classification_report(dev_y,neigh.predict(dev_x),target_names = None))

              precision    recall  f1-score   support

         2.0       0.77      0.76      0.76       138
         3.0       0.70      0.90      0.79       372
         5.0       0.73      0.43      0.54       100
         6.0       0.98      0.98      0.98       168
         8.0       0.70      0.67      0.68       192
        10.0       0.83      0.47      0.60       142
        11.0       0.94      0.97      0.96        79
        12.0       0.97      0.98      0.98       128
        14.0       1.00      1.00      1.00        66

   micro avg       0.80      0.80      0.80      1385
   macro avg       0.85      0.80      0.81      1385
weighted avg       0.81      0.80      0.79      1385



效果不如SVC